In [ ]:
"""
AUTHOR : VIRAJ

DATE CREATED : (Saturday) 2nd October 2021

DESC: 

"""

In [253]:
import pandas as pd
from datetime import datetime

In [270]:
receipts = pd.read_json('receipts.json',lines=True)
receipts.info()
#receipts.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
_id                        1119 non-null object
bonusPointsEarned          544 non-null float64
bonusPointsEarnedReason    544 non-null object
createDate                 1119 non-null object
dateScanned                1119 non-null object
finishedDate               568 non-null object
modifyDate                 1119 non-null object
pointsAwardedDate          537 non-null object
pointsEarned               609 non-null float64
purchaseDate               671 non-null object
purchasedItemCount         635 non-null float64
rewardsReceiptItemList     679 non-null object
rewardsReceiptStatus       1119 non-null object
totalSpent                 684 non-null float64
userId                     1119 non-null object
dtypes: float64(4), object(11)
memory usage: 131.2+ KB


In [362]:
def date_converter(x):
    '''
    Date provided is in UTC Format and we use this function to convert it to understandable format
    
    INPUTS
        x : Date string in UTC Format
    
    OUTPUTS
        y : Return Date in datetime format
    
    '''
    
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return(None)

In [272]:
# set().union(*(d.keys() for d in receipts['_id']))
# set().union(*(d.keys() for d in receipts['createDate']))
# set().union(*(d.keys() for d in receipts['dateScanned']))
# set().union(*(d.keys() for d in receipts.dropna(subset=['finishedDate'])['finishedDate']))
# set().union(*(d.keys() for d in receipts.dropna(subset=['modifyDate'])['modifyDate']))
# set().union(*(d.keys() for d in receipts.dropna(subset=['pointsAwardedDate'])['pointsAwardedDate']))
# set().union(*(d.keys() for d in receipts.dropna(subset=['purchaseDate'])['purchaseDate']))


In [273]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
receipts['createDate'] = receipts['createDate'].apply(lambda x: date_converter(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: date_converter(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: date_converter(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: date_converter(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: date_converter(x))

In [363]:
receipts.head(2)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052


In [275]:

l = []
for i in range(len(receipts)):
    try:
        l.append(receipts['rewardsReceiptItemList'].iloc[i][0])
    except:
        #print(receipts['rewardsReceiptItemList'].iloc[i])
        pass


In [276]:
rewards_receipts_item = pd.DataFrame(l)
rewards_receipts_item["rewardsReceiptStatus"] = receipts["rewardsReceiptStatus"]
rewards_receipts_item['receipts_id']= receipts["_id"]
rewards_receipts_item.sample()

,barcode,brandCode,competitiveProduct,competitorRewardsGroup,deleted,description,discountedItemPrice,finalPrice,itemNumber,itemPrice,...,rewardsGroup,rewardsProductPartnerId,targetPrice,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,rewardsReceiptStatus,receipts_id
409,043000000342,NaN,NaN,NaN,NaN,"GENERAL FOODS INTERNATIONAL Vanilla Creme, No Caffeine, 9-Ounce Units (Pack of 6)",NaN,1,NaN,1,...,MAXWELL HOUSE INTERNATIONAL INSTANT COFFEE,559c2234e4b06aca36af13c6,800,NaN,NaN,NaN,NaN,NaN,FINISHED,6009e72c0a720f0535000003


In [277]:
import numpy as np
rewards_receipts_item['rewards_id'] = np.zeros((len(rewards_receipts_item),1))
for i in range(len(rewards_receipts_item)):
    rewards_receipts_item['rewards_id'].iloc[i]= i+1

C:\Users\viraj\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [278]:
col_val = ['rewards_id','receipts_id','rewardsReceiptStatus',
           'barcode', 'brandCode', 'competitiveProduct', 'competitorRewardsGroup',
       'deleted', 'description', 'discountedItemPrice', 'finalPrice',
       'itemNumber', 'itemPrice', 'metabriteCampaignId', 'needsFetchReview',
       'needsFetchReviewReason', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'originalMetaBriteQuantityPurchased',
       'originalReceiptItemText', 'partnerItemId', 'pointsEarned',
       'pointsNotAwardedReason', 'pointsPayerId', 'preventTargetGapPoints',
       'priceAfterCoupon', 'quantityPurchased', 'rewardsGroup',
       'rewardsProductPartnerId', 'targetPrice', 'userFlaggedBarcode',
       'userFlaggedDescription', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity']
rewards_receipts = rewards_receipts[col_val]
rewards_receipts.head()

,rewards_id,receipts_id,rewardsReceiptStatus,barcode,brandCode,competitiveProduct,competitorRewardsGroup,deleted,description,discountedItemPrice,...,priceAfterCoupon,quantityPurchased,rewardsGroup,rewardsProductPartnerId,targetPrice,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
0,1.0,5ff1e1eb0a720f0523000575,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,5.0,NaN,NaN,NaN,4011,NaN,True,26.00,5.0
1,2.0,5ff1e1bb0a720f052300056b,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,5ff1e1f10a720f052300057a,REJECTED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4011,NaN,True,26.00,3.0
3,4.0,5ff1e1ee0a7214ada100056f,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,4.0,NaN,NaN,NaN,4011,NaN,True,28.00,4.0
4,5.0,5ff1e1d20a7214ada1000561,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
receipts=receipts.drop(['rewardsReceiptItemList'], axis=1)

In [280]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
_id                        1119 non-null object
bonusPointsEarned          544 non-null float64
bonusPointsEarnedReason    544 non-null object
createDate                 1119 non-null object
dateScanned                1119 non-null object
finishedDate               568 non-null object
modifyDate                 1119 non-null object
pointsAwardedDate          537 non-null object
pointsEarned               609 non-null float64
purchaseDate               671 non-null object
purchasedItemCount         635 non-null float64
rewardsReceiptStatus       1119 non-null object
totalSpent                 684 non-null float64
userId                     1119 non-null object
dtypes: float64(4), object(10)
memory usage: 122.5+ KB


In [281]:
receipts.to_csv('receiptstosql.csv')

In [282]:
rewards_receipts_item.to_csv('rewards_receiptstosql.csv')

## Users Table Processing

In [283]:
#users processing
users = pd.read_json('users.json',lines=True)
users.info()
users.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
_id             495 non-null object
active          495 non-null bool
createdDate     495 non-null object
lastLogin       433 non-null object
role            495 non-null object
signUpSource    447 non-null object
state           439 non-null object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [284]:
set().union(*(d.keys() for d in users['_id']))
set().union(*(d.keys() for d in users['createdDate']))
set().union(*(d.keys() for d in users.dropna(subset=['lastLogin'])['lastLogin']))#not all lastlogin have date

{'$date'}

In [285]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])

In [286]:
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))

In [287]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


In [288]:
users.to_csv('userstosql.csv')

## Brands Table Processing

In [289]:
l1 = []
for i in range(len(brands)):
    try:
        l1.append(brands['cpg'].iloc[i])
    except:
        #print(brands['cpg'].iloc[i])
        pass
df_cpg = pd.DataFrame(l1)
df_cpg.head()

""


In [290]:
brands = pd.read_json('brands.json',lines=True)
brands.info()
brands.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
_id             1167 non-null object
barcode         1167 non-null int64
brandCode       933 non-null object
category        1012 non-null object
categoryCode    517 non-null object
cpg             1167 non-null object
name            1167 non-null object
topBrand        555 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 73.0+ KB


,_id,barcode,brandCode,category,categoryCode,cpg,name,topBrand
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,NaN,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'}",test brand @1612366101024,0.0
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,STARBUCKS,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, '$ref': 'Cogs'}",Starbucks,0.0
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146176,0.0
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146051,0.0
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, '$ref': 'Cogs'}",test brand @1612366146827,0.0


In [291]:
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])
brands['cpg_id']=brands['cpg'].apply(lambda x: x['$id']['$oid'])
brands['cpg_ref']=brands['cpg'].apply(lambda x: x['$ref'])

In [292]:
brands.head()

,_id,barcode,brandCode,category,categoryCode,cpg,name,topBrand,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'}",test brand @1612366101024,0.0,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, '$ref': 'Cogs'}",Starbucks,0.0,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146176,0.0,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146051,0.0,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, '$ref': 'Cogs'}",test brand @1612366146827,0.0,5332fa12e4b03c9a25efd1e7,Cogs


In [293]:
brands=brands.drop(['cpg'], axis=1)
brands.head()

,_id,barcode,brandCode,category,categoryCode,name,topBrand,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,test brand @1612366101024,0.0,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,Starbucks,0.0,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,test brand @1612366146176,0.0,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,test brand @1612366146051,0.0,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,5332fa12e4b03c9a25efd1e7,Cogs


In [294]:
brands['name']  = brands['name'].apply(lambda x: str(x))
brands['cpg_ref']  = brands['cpg_ref'].apply(lambda x: str(x))
brands['brandCode']  = brands['brandCode'].apply(lambda x: str(x))

In [295]:
brands_new.to_csv('brandstosql.csv')

In [338]:
brands.sample()

,_id,barcode,brandCode,category,categoryCode,name,topBrand,cpg_id,cpg_ref
654,5f35a0babe37ce6853cff1fe,511111915355,TEST BRANDCODE @1597350074237,Baking,BAKING,test brand @1597350074236,NaN,5f35a0babe37ce6853cff1fd,Cogs


In [310]:
users.sample()

,_id,active,createdDate,lastLogin,role,signUpSource,state
138,5ffc9d9f04929111f6e92456,True,2021-01-11 18:49:03,2021-01-11 18:49:03,consumer,Email,WI


In [311]:
rewards_receipts_item.sample()

,barcode,brandCode,competitiveProduct,competitorRewardsGroup,deleted,description,discountedItemPrice,finalPrice,itemNumber,itemPrice,...,rewardsProductPartnerId,targetPrice,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,rewardsReceiptStatus,receipts_id,rewards_id
49,046704085905,NaN,NaN,NaN,NaN,T.G.I. FRIDAY'S(r) Frozen Loaded Baked Potato Cheese Spread DIP BOX 8 OZ 004670408590,NaN,4.55,NaN,4.55,...,559c2234e4b06aca36af13c6,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,5ff36c6d0a720f0523000597,50.0


In [313]:
receipts.sample()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
1091,603c8cdf0a7217c72c0003bb,NaN,NaN,2021-03-01 06:42:39,2021-03-01 06:42:39,None,2021-03-01 06:42:39,None,NaN,None,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33


# Data Quality Issue

### Check Duplicate Users

In [302]:
users_after_all_columns_duplicate_removal = users.drop_duplicates()

In [305]:
unique_users, total_users = (len(users_after_all_columns_duplicate_removal),len(users))

In [306]:
print('Total Number of Duplicate(All the columns) Users:',total_users-unique_users)

Total Number of Duplicate(All the columns) Users: 283


In [307]:
users_after_all_ids_duplicate_removal = users['_id'].drop_duplicates()

In [308]:
unique_users, total_users = (len(users_after_all_ids_duplicate_removal),len(users))

In [309]:
print('Total Number of Duplicate(IDS) Users:',total_users-unique_users)

Total Number of Duplicate(IDS) Users: 283


### Check If we have missing prices in Receipts

In [312]:
rewards_receipts_item.columns

Index(['barcode', 'brandCode', 'competitiveProduct', 'competitorRewardsGroup',
       'deleted', 'description', 'discountedItemPrice', 'finalPrice',
       'itemNumber', 'itemPrice', 'metabriteCampaignId', 'needsFetchReview',
       'needsFetchReviewReason', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'originalMetaBriteQuantityPurchased',
       'originalReceiptItemText', 'partnerItemId', 'pointsEarned',
       'pointsNotAwardedReason', 'pointsPayerId', 'preventTargetGapPoints',
       'priceAfterCoupon', 'quantityPurchased', 'rewardsGroup',
       'rewardsProductPartnerId', 'targetPrice', 'userFlaggedBarcode',
       'userFlaggedDescription', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'rewardsReceiptStatus', 'receipts_id',
       'rewards_id'],
      dtype='object')

In [318]:
rewards_receipts_item[['itemPrice','targetPrice','finalPrice']].head()

,itemPrice,targetPrice,finalPrice
0,26.00,NaN,26.00
1,1,NaN,1
2,NaN,NaN,NaN
3,28.00,NaN,28.00
4,1,NaN,1


In [317]:
(rewards_receipts_item['itemPrice'].astype(float) - rewards_receipts_item['finalPrice'].astype(float)).dropna().sum()

0.0

This shows that the columns 'itemPrice' and 'finalPrice' have exactly same data. Lets check if we have any price set to Nan

In [319]:
rewards_receipts_item['finalPrice'].isna().sum()

14

In [324]:
print('Out of ',len(rewards_receipts_item),'Samples,',rewards_receipts_item['finalPrice'].isna().sum(),'samples have NaN as Final Price')

Out of  679 Samples, 14 samples have NaN as Final Price


### Check if we have missing total spent amount from receipts

In [326]:
receipts.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId'],
      dtype='object')

In [335]:
print('Out of ',len(receipts),'Samples,',receipts['totalSpent'].isna().sum(),'samples have NaN as totalSpent')

Out of  1119 Samples, 435 samples have NaN as totalSpent


### Check if we have unique barcodes

In [339]:
brands.columns

Index(['_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'name',
       'topBrand', 'cpg_id', 'cpg_ref'],
      dtype='object')

Bar Code of all the products must be unique

In [340]:
brands_temp = brands[brands['barcode'].notna()]

In [342]:
brands_removing_duplicate_barcodes =brands_temp['barcode'].drop_duplicates()

In [343]:
unique_users, total_users = (len(brands_removing_duplicate_barcodes),len(brands_temp))

In [344]:
print('Total Number of Duplicate BarCodes:',total_users-unique_users)

Total Number of Duplicate BarCodes: 7


We have 7 Duplicate Barcodes, which might cause an issue


### Mactching Barcode and brandcode from Brands to that of  rewards_receipts_item

In [345]:
rewards_receipts_item.columns

Index(['barcode', 'brandCode', 'competitiveProduct', 'competitorRewardsGroup',
       'deleted', 'description', 'discountedItemPrice', 'finalPrice',
       'itemNumber', 'itemPrice', 'metabriteCampaignId', 'needsFetchReview',
       'needsFetchReviewReason', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'originalMetaBriteQuantityPurchased',
       'originalReceiptItemText', 'partnerItemId', 'pointsEarned',
       'pointsNotAwardedReason', 'pointsPayerId', 'preventTargetGapPoints',
       'priceAfterCoupon', 'quantityPurchased', 'rewardsGroup',
       'rewardsProductPartnerId', 'targetPrice', 'userFlaggedBarcode',
       'userFlaggedDescription', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'rewardsReceiptStatus', 'receipts_id',
       'rewards_id'],
      dtype='object')

In [347]:
brands.columns

Index(['_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'name',
       'topBrand', 'cpg_id', 'cpg_ref'],
      dtype='object')

In [358]:
uniques_barcodes_in_receipts_items = set(rewards_receipts_item['brandCode'].dropna())
uniques_barcodes_in_brands = set(brands['brandCode'].dropna())

In [365]:
intersection = uniques_barcodes_in_receipts_items.intersection(uniques_barcodes_in_brands)
a = len(uniques_barcodes_in_receipts_items - intersection)

In [366]:
b = len(uniques_barcodes_in_receipts_items)

In [361]:
len(uniques_barcodes_in_brands)

898

In [368]:
print('Out of',str(b),'samples,',str(a),'smaples do naot have a match in the brands table')

Out of 18 samples, 12 smaples do naot have a match in the brands table


Also, the barcodes present in the table 'rewards_receipts_item' do not have a single match with barcodes in table 'brands'